# Python Decorators Tutorials

## 1. What is Python Wrappers
- Python wrappers are functions that are added to another function which then can add additional functionality or modifies its behavior without directly changing its source code. 

- Example 1: Without `functools.wraps()`
    - We can observe that both the `first_function` and `second_function` after applying the `a_decorator` have the same `__name__` and `__doc__` string which is `name='wrapper', doc='A wrapper function'`
- Ideally, it should show the name and docstring of wrapped function (`func`) instead of wrapping function `wrapper`.

In [28]:
def a_decorator(func):
    def wrapper(*args, **kwargs):
        """A wrapper function"""
        # Extend some capabilities of func
        func()
    return wrapper
 
@a_decorator
def first_function():
    """This is docstring for first function"""
    print("first function")
 
@a_decorator
def second_function(a):
    """This is docstring for second function"""
    print("second function")
 
print(f"First Function : name='{first_function.__name__}', doc='{first_function.__doc__}'")
print(f"Second Function: name='{second_function.__name__}', doc='{second_function.__doc__}'")

First Function : name='wrapper', doc='A wrapper function'
Second Function: name='wrapper', doc='A wrapper function'


- `functools.wraps()` as decorator to wrapper function
    - Now, the function names and docstrings of wrapped function have been display correctly

In [30]:
import functools
def a_decorator(func):
    @functools.wraps(func)
    def wrapper(*args, **kwargs):
        """A wrapper function"""
        func()
    return wrapper
 
@a_decorator
def first_function():
    """This is docstring for first function"""
    print("first function")
 
@a_decorator
def second_function(a):
    """This is docstring for second function"""
    print("second function")
 
print(f"First Function : name='{first_function.__name__}', doc='{first_function.__doc__}'")
print(f"Second Function: name='{second_function.__name__}', doc='{second_function.__doc__}'")

First Function : name='first_function', doc='This is docstring for first function'
Second Function: name='second_function', doc='This is docstring for second function'


## 2. Examples of Wrapper Function

### 1.1 Timer
- To create the **decorator** in Python, we need to define a function called `timer` that takes a parameter called `func` = *a decorator function*. 
- Inside the `timer` function, we define another function called `wrapper` that takes the **arguments** typically *passed to the function we want to decorate*.
- Within the `wrapper` function, we invoke the desired function using the provided arguments. 
    - We can do this with the line: `result = func(*args, **kwargs)`.
    - Finally, the `wrapper` function returns the **result of the decorated function’s execution**. 
- To utilize the decorator, you can apply it to the desired function using the @ symbol.

In [15]:
import functools
import time

# Timeit decorators
def timer(func):
    @functools.wraps(func)
    def wrapper(*args, **kwargs):
        """This is timeit's wrapper func"""
        start = time.perf_counter()
        # call the decorated function
        result = func(*args, **kwargs)
        end = time.perf_counter()
        print(f"Function '{func.__name__}' took {end - start:.6f} seconds to complete")
        return result

    return wrapper

In [16]:
@timer
def train_model(a, b):
    # simulate a function execution by pausing the program for 2 seconds
    time.sleep(2) 
    print(a*b)


train_model(1000, b=10) 

10000
Function 'train_model' took 2.001211 seconds to complete


### 1.2. Debugger
- An additional useful wrapper function can be created to facilitate debugging by printing the inputs and outputs of each function. 
- This approach allows us to gain insight into the execution flow of various functions without cluttering our applications with multiple print statements.

In [31]:
def debug(func):
    @functools.wraps(func)
    def wrapper(*args, **kwargs):
        # print the fucntion name and arguments
        print(f"Calling '{func.__name__}' with args: {args} kwargs: {kwargs}")
        # call the function
        result = func(*args, **kwargs)
        # print the results
        print(f"'{func.__name__}' returned: {result}")
        return result
    return wrapper

In [22]:
@debug
def add_numbers(x, y):
    return x + y
add_numbers(7, y=5)

Calling 'add_numbers' with args: (7,) kwargs: {'y': 5}
'add_numbers' returned: 12


12

### 1.3. Exception Handler
- The exception_handler the wrapper will catch any exceptions raised within the decorator function

In [33]:
def exception_handler(func):
    @functools.wraps(func)
    def wrapper(*args, **kwargs):
        try: 
            return func(*args, **kwargs)
        except Exception as e:
            # Handle the exception
            print(f"An exception occurred: {str(e)}")
            # Optionally, perform additional error handling or logging
            # Reraise the exception if needed
    return wrapper

In [34]:
@exception_handler
def divide(x, y):
    return x / y
divide(10, 0)

An exception occurred: division by zero


### 1.4. Call Count

In [44]:
# Call Count
def count_call(func):
    @functools.wraps(func)
    def wrapper(*args, **kwargs):
        wrapper.count += 1
        result = func(*args, **kwargs)
        print(f"{func.__name__} has been called {wrapper.count} times")
        return result

    wrapper.count = 0  # initialise to 0 before you ran any of them.
    return wrapper

In [53]:
@count_call
def add_numbers(a, b):
    """This is plus_two_number function"""
    return a + b

print(add_numbers(1000, 999))
print(add_numbers(1, 2))

add_numbers has been called 1 times
1999
add_numbers has been called 2 times
3


### 1.5. Built-in `lru_cache`

- `@fucntools.lru_cache`: When calling the input function,
    - It first checks if its arguments are present in the cache.
    - If it’s the case, return the result.
    - Otherwise, compute it and put it in the cache

In [60]:
@timer
@functools.lru_cache
def multiply_numbers(a, b):
    """This is plus_two_number function"""
    return a * b

print(multiply_numbers(99999, 9876541))
print(multiply_numbers(1, 2))
print(multiply_numbers(99999, 9876541))

Function 'multiply_numbers' took 0.000002 seconds to complete
987644223459
Function 'multiply_numbers' took 0.000002 seconds to complete
2
Function 'multiply_numbers' took 0.000001 seconds to complete
987644223459


- As you can see that `multiply_numbers` with input` a=99999, b=9876541` called 2 times
    - For the first time, it took 0.000002 to complete
    - For the second time, it only took 0.000001 to complete this is because the result from the second time is retrieved from the cache thanks to the `@functools.lru_cache`


### 1.6. Retry
- This wrapper retries the execution of a function a specified number of times with a delay between retries.

In [41]:
import logging
import traceback

LOG_FORMAT = "%(asctime)s - %(levelname)s - %(pathname)s - %(funcName)s - %(lineno)d -msg: %(message)s"
logging.basicConfig(level=logging.INFO, format=LOG_FORMAT)

def retry(max_attempts, delay):
    """
    retry help decorator.
    :param max_attempts: the retry num; retry sleep sec
    :return: decorator
    """
    def decorator(func):
        """decorator"""
        @functools.wraps(func)   # preserve information about the original function, or else the wrapped func name will be "wrapper" not "func"
        def wrapper(*args, **kwargs):
            """wrapper"""
            for attempt in range(max_attempts):
                try:
                    return func(*args, **kwargs) 
                except Exception as err: 
                    logging.error(err)
                    logging.error(traceback.format_exc())
                    time.sleep(delay)
                logging.error(f"Trying attempt {attempt+1} of {max_attempts}")
            logging.error(f"func {func.__name__}retry failed")
            raise Exception(f'Exceed max retry num: {max_attempts} failed')

        return wrapper

    return decorator

In [42]:
@retry(max_attempts=2, delay=2)
def fetch_data(url):
    print("Fetching the data..")
    # raise timeout error to simulate a server not responding..
    raise TimeoutError("Server is not responding.")

fetch_data("https://example.com/data")

2023-10-18 00:08:52,201 - ERROR - /var/folders/ww/280v33ws1pdd58c895ntxf4w0000gn/T/ipykernel_48486/3869183515.py - wrapper - 22 -msg: Server is not responding.
2023-10-18 00:08:52,212 - ERROR - /var/folders/ww/280v33ws1pdd58c895ntxf4w0000gn/T/ipykernel_48486/3869183515.py - wrapper - 23 -msg: Traceback (most recent call last):
  File "/var/folders/ww/280v33ws1pdd58c895ntxf4w0000gn/T/ipykernel_48486/3869183515.py", line 20, in wrapper
    return func(*args, **kwargs)
          ^^^^^^^^^^^^^^^^^^^^^
  File "/var/folders/ww/280v33ws1pdd58c895ntxf4w0000gn/T/ipykernel_48486/579690303.py", line 5, in fetch_data
    raise TimeoutError("Server is not responding.")
TimeoutError: Server is not responding.



Fetching the data..


2023-10-18 00:08:54,218 - ERROR - /var/folders/ww/280v33ws1pdd58c895ntxf4w0000gn/T/ipykernel_48486/3869183515.py - wrapper - 25 -msg: Trying attempt 1 of 2
2023-10-18 00:08:54,219 - ERROR - /var/folders/ww/280v33ws1pdd58c895ntxf4w0000gn/T/ipykernel_48486/3869183515.py - wrapper - 22 -msg: Server is not responding.
2023-10-18 00:08:54,221 - ERROR - /var/folders/ww/280v33ws1pdd58c895ntxf4w0000gn/T/ipykernel_48486/3869183515.py - wrapper - 23 -msg: Traceback (most recent call last):
  File "/var/folders/ww/280v33ws1pdd58c895ntxf4w0000gn/T/ipykernel_48486/3869183515.py", line 20, in wrapper
    return func(*args, **kwargs)
          ^^^^^^^^^^^^^^^^^^^^^
  File "/var/folders/ww/280v33ws1pdd58c895ntxf4w0000gn/T/ipykernel_48486/579690303.py", line 5, in fetch_data
    raise TimeoutError("Server is not responding.")
TimeoutError: Server is not responding.



Fetching the data..


2023-10-18 00:08:56,222 - ERROR - /var/folders/ww/280v33ws1pdd58c895ntxf4w0000gn/T/ipykernel_48486/3869183515.py - wrapper - 25 -msg: Trying attempt 2 of 2
2023-10-18 00:08:56,223 - ERROR - /var/folders/ww/280v33ws1pdd58c895ntxf4w0000gn/T/ipykernel_48486/3869183515.py - wrapper - 26 -msg: func fetch_dataretry failed


Exception: Exceed max retry num: 2 failed